__Задача 1__

Используя данные о хоккеистах (скачать можно [здесь](https://habr.com/ru/post/301340/)), проверьте, является ли среди хоккеистов из Финляндии, Норвегии и Дании значимым отличие роста. В качестве ответа запишите 1, если является, и 0 иначе.

_Подсказка_: не забудьте предобработать данные (например, дублируются ли данные о конкретных игроках?).


In [10]:
import numpy as np
import pandas as pd
from scipy import stats

In [4]:
df = pd.read_csv('hockey_players.csv')
df.head()

,year,country,no,name,position,side,height,weight,birth,club,age,cohort,bmi
0,2001,RUS,10,tverdovsky oleg,D,L,185,84.0,1976-05-18,anaheim mighty ducks,24.952772,1976,24.543462
1,2001,RUS,2,vichnevsky vitali,D,L,188,86.0,1980-03-18,anaheim mighty ducks,21.119781,1980,24.332277
2,2001,RUS,26,petrochinin evgeni,D,L,182,95.0,1976-02-07,severstal cherepovetal,25.229295,1976,28.680111
3,2001,RUS,28,zhdan alexander,D,R,178,85.0,1971-08-28,ak bars kazan,29.675565,1971,26.827421
4,2001,RUS,32,orekhovsky oleg,D,R,175,88.0,1977-11-03,dynamo moscow,23.490760,1977,28.734694


In [5]:
df['country'].unique()

array(['RUS', 'AUT', 'BLR', 'CAN', 'CZE', 'FIN', 'GER', 'ITA', 'JPN',
       'LAT', 'NOR', 'SUI', 'SVK', 'SWE', 'UKR', 'USA', 'POL', 'SLO',
       'DEN', 'FRA', 'KAZ', 'HUN'], dtype=object)

Избавимся от дубликатов

In [6]:
df = df.drop_duplicates(subset=['name', 'birth'], keep='last')

Выделим рассматриваемые признаки, разбив на выборки по значению категории страна: "FIN", "NOR", "DEN":

In [7]:
heights = [df.loc[df['country'] == country, 'height'] for country in ['FIN', 'NOR', 'DEN']]
bmis = [df.loc[df['country'] == country, 'bmi'] for country in ['FIN', 'NOR', 'DEN']]

Подсчитаем F-статистики для роста и BMI:

In [8]:
def f_stat(ys):
    y = np.concatenate(ys)

    ss_b = sum((yi.mean() - y.mean()) ** 2 * yi.size for yi in ys)
    ss_w = sum(((yi - yi.mean()) ** 2).sum() for yi in ys)
    
    sigma_b = ss_b / (len(ys) - 1)
    sigma_w = ss_w / (y.size - len(ys))
    
    return sigma_b / sigma_w

In [9]:
f_stat_height = f_stat(heights)
f_stat_bmi = f_stat(bmis)

f_stat_height, f_stat_bmi

(3.8026212813131024, 5.918577060155501)

Для построения критической области нам нужны квантили распределения Фишера с параметрами $k_1 = k - 1$, $k_2 = n - k$, где $k$ — число уровней, $n$ — общее число элементов во всех выборках.


In [25]:
k = len(bmis)
n = bmis[0].size + bmis[1].size + bmis[2].size
k1 = k - 1
k2 = n - k
k1, k2

(2, 430)

In [26]:
alpha = 0.05
t = stats.f.ppf(1 - alpha, k1, k2)
t

3.016700272452327

Критическая область: \
$\Omega_\alpha = \left( 3.017, \infty \right)$\
И значения $F$ статистики для роста и BMI, равные $\f_stat_height\$, $\f_stat_bmi$ и  соответственно, в нее попадают. Нулевая гипотеза отвергается, следовательно является значимым отличие в росте и BMI хоккеистов из Финляндии, Норвегии и Дании.
